In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

The following GPU devices are available: /device:GPU:0


**Note** : if you encounter an error running any of the examples below consider restarting the kernel and running this cell first

# DeepSIBA example 1 : Train ensemble
In this example a deepSIBA ensemble model will be trained from scratch using the model_params and train_params dictionaries

In [2]:
model_params = {
    "max_atoms" : int(70), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.2,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.2, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 ,"ConGauss":True
}

The model_params dictionary contains the parameters to build the deepSIBA siamese GCN architecture, more specifically:
1. **max_atoms, num_atom_features, max_degree and num_bond_features** refer to the parameters needed to featurize the input chemical structures. For more information, refer to the *ESI of the deepSIBA publication*.
2. **graph_conv_width, conv1d_filters, conv1d_size, dropout_encoder** refer to the parameters of the siamese graph encoders.
3. **conv1d_filters_dist, conv1d_size_dist, dropout_dist, pool_size, dense_size, l2reg** refer to the parameters of the distance module.
4. **dist_thresh** is the distance threshold to consider 2 chemical structures similar in biological effect (needed for custom training metrics).
5. **lr** is the learning rate.
6. **ConGauss** is by default set to False. **Set it True, only if training becomes difficult due to Loss becoming frequently Inf.** If set to True, a Gaussian Layer constrained to 0 to 1 is used instead of the original Gaussian layer.

In [3]:
train_params = {
    "cell_line" : "merged_go", "split" : "5_fold_cv_split", "number_folds" : [0],
    "output_dir" : "/home/biolab/Documents/Go distances/deepSIBA/results",
    "batch_size" : int(256), "epochs" : int(10), 
    "N_ensemble" : int(1), "nmodel_start" : int(0), "prec_threshold" : 0.2,
    "test_value_norm" : False,
    "predict_batch_size":int(2048)
}

The train_params dictionary contains the parameters required to train deepSIBA:
1. **cell_line** is the cellular model of choice out of **(a375,pc3,vcap,mcf7,merged)** for which we have enough available data. The merged option refers to data merged across cell lines.
2. **split** is one of **(train_test_split,5_fold_cv_split,alldata)**. The data to train the models are available in our google drive folder, see **data/readme.md**.
3. **number_folds** is a list, if split == train_test_split the number_folds should be [0]. If the split is a 5_fold_cv_split the number_folds should be [0,1,2,3,4] in order to train the model in all splits. If you want to train a model on a specific fold, e.g. the 3rd one, the number_folds should be [2].
4. **output_dir** is the full path to the specified output directory.
5. **N_ensemble** is the number of models to train and include in the ensemble.
6. **nmodel_start** this should be set to 0 if training for the first time, but if training is halted, nmodel_start specifies the model number in the ensemble to start training from.
7. **prec_threshold** is the distance threshold to consider 2 chemical structures similar in biological effect (needed for custom training metrics).
8. **test_value_norm** is the test/val value already normalized between 0-1.
9. **predict_batch_size** is the batch size with which predictions for the pairs of the validation set are made.

In [4]:
from deepSIBA_train import siba_trainer
example_1 = siba_trainer(train_params, model_params)

Using TensorFlow backend.
/home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
W0504 14:59:48.481312 140248729470784 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0504 14:59:48.486895 140248729470784 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0504 14:59:48.487732 140248729470784 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is dep

Epoch 1/10
14787/14787 [==============================] - 1153s 78ms/step - loss: -1.6947 - mean_squared_error: 0.0134 - get_cindex: 0.7306 - r_square: 0.4023 - pearson_r: 0.6543 - mse_similars: nan
Epoch 2/10
14787/14787 [==============================] - 1153s 78ms/step - loss: -2.1678 - mean_squared_error: 0.0060 - get_cindex: 0.8206 - r_square: 0.7327 - pearson_r: 0.8622 - mse_similars: nan
Epoch 3/10
14787/14787 [==============================] - 1153s 78ms/step - loss: -2.2939 - mean_squared_error: 0.0049 - get_cindex: 0.8394 - r_square: 0.7824 - pearson_r: 0.8895 - mse_similars: nan
Epoch 4/10
14787/14787 [==============================] - 1152s 78ms/step - loss: -2.3522 - mean_squared_error: 0.0045 - get_cindex: 0.8481 - r_square: 0.7992 - pearson_r: 0.8986 - mse_similars: 0.0014
Epoch 5/10
14787/14787 [==============================] - 1152s 78ms/step - loss: -2.3876 - mean_squared_error: 0.0043 - get_cindex: 0.8537 - r_square: 0.8084 - pearson_r: 0.9038 - mse_similars: 0.0013

# DeepSIBA example 2 : Load trained ensemble and predict
In this example a trained deepSIBA ensemble model will be loaded and used to make predictions for the appropriate test set.
For each of the cell lines, trained ensembles of either 50 or 10 models for all available splits, can be found in our google drive, see **trained_models/readme.md**.

In [4]:
model_params = {
    "max_atoms" : int(70), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.2,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.2, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 ,"ConGauss":True
}

First of all the model is compiled with the parameters (model_params) as described in example 1.

In [5]:
from deepSIBA_model import siamese_model,enc_graph
siamese_net=siamese_model(model_params)
print(siamese_net.summary())

Using TensorFlow backend.
W0504 18:56:56.373223 139938392590144 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0504 18:56:56.378782 139938392590144 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0504 18:56:56.379462 139938392590144 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0504 18:56:56.492802 139938392590144 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs_1 (InputLayer)      (None, 70, 62)       0                                            
__________________________________________________________________________________________________
bond_inputs_1 (InputLayer)      (None, 70, 5, 6)     0                                            
__________________________________________________________________________________________________
edge_inputs_1 (InputLayer)      (None, 70, 5)        0                                            
__________________________________________________________________________________________________
atom_inputs_2 (InputLayer)      (None, 70, 62)       0                                            
__________________________________________________________________________________________________
bond_input

<img src="architecture.png">

In [6]:
test_params = {
    "cell_line" : "merged_go", "split" : "5_fold_cv_split", "fold_id" : int(0),
    "N_ensemble" : int(1), "prec_threshold" : 0.2,
    "name_pattern":"siam_no_augment",
    "test_value_norm" : False,
    "predict_batch_size":2048,
    "to_load": 2,
    "mu_path":'/home/biolab/Documents/Go distances/deepSIBA/results/cold/mu/',
    "sigma_path":'/home/biolab/Documents/Go distances/deepSIBA/results/cold/sigma/',
    "prediction_pattern":["cold_mu_","cold_sigma_"]
}

The test_params dictionary contains the parameters required to train deepSIBA:
1. **cell_line** is the cellular model of choice out of **(a375,pc3,vcap,mcf7)** for which we have enough available data. Later a merged option will be added.
2. **split** is one of **(train_test_split,5_fold_cv_split)**. 
3. **fold_id** is an integer, if split == train_test_split the fold_id should be 0. If the split is a 5_fold_cv_split the fold_id should be 0,1,2,3 or 4 (one less than the corresponding folder's name for this fold) in order to test the model's performance in **a specific split**.
4. **N_ensemble** is the number of total already trained models and at the same time the models included in the ensembled prediction.
5. **prec_threshold** is the distance threshold to consider 2 chemical structures similar in biological effect (needed for custom training metrics).
6. **name_pattern** is the pattern of the name of files of models' saved weights. **For example** if the weights are saved in files with names such as **siam_no_augment_18.h5** the **pattern is siam_no_augment** .
7. **test_value_norm** is the test/val value already normalized between 0-1.
8. **predict_batch_size** is the batch size with which predictions for the pairs of the validation set are made.
9. **to_load** defines whether trained **models are loaded (1)** and predictions are made or if the **predictions of mean and sigma are loaded directly (give a random integer different from 1).**
10. **mu_path** is the path where the mean of distance predictions of each model are. If to_load=1 then its value doesn't matter.
11. **sigma_path** is the path where the standard deviation of distance predictions of each model are. If to_load=1 then its value doesn't matter.
12. **prediction_pattern** is a list with the pattern of the filenames of mean (mu) and sigma of each model. First is the mean and second the sigma. **THE PREDICTIONS MUST BE SAVED AS .npy files**. **For example** if the pattern is "cold_mu_" then the files are like "cold_mu_**x**.npy" where x is the model's number

**NOTE :** The saved model weights are in the subfolders of "trained_models/" and their exact position is described fully given **cell_line** and **split** parameters.


In [7]:
from deepSIBA_ensembles import siba_val_loader
df_cold=siba_val_loader(test_params, model_params,siamese_net)

In [ ]:
df_cold

Get the ensembled predictions with siba_val_loader function.
The data frame with the predictions and the corresponding CV (coefficient of variation) of each prediction is presented below

**NOTE:** All distance values have been adjusted in the range 0 to 1.

Finally the model's performance together with a scatterplot of the predicted values VS the true values of the test set, are given bellow:

**NOTE:** If there are no predictions lower than the precision threshold defined, the MSE in the predictions lower than the defined threshold and the precision will be **None**

In [8]:
from utility.evaluator import model_evaluate
import numpy as np
from matplotlib import pyplot as plt
true = np.array(df_cold.value)
pred = np.array(df_cold.mu)
get_eval=model_evaluate(pred,true,test_params["prec_threshold"],df_cold)
print(get_eval)
plt.scatter(pred,true,s = 0.5, label = "pearson`s r: "+str(round(get_eval['cor'][0],2)))
plt.xlabel("predicted GO distance")
plt.ylabel("True GO distance")
plt.legend(loc='upper left')
plt.show()

        cor   mse_all  mse_similars  precision  accuracy       FPR  positives
0  0.498681  0.017472      0.029148   0.300826  0.898095  0.000572        605


<Figure size 640x480 with 1 Axes>

# DeepSIBA example 3 : Screening

In this example a trained deepSIBA ensemble model will be used to identify chemical structures from the Chembl and the CMap datasets that affect similar biological processes to a query structure. 
Given a **query chemical structure and a cellular model**:

1. If a structural analogue to the query (ECFP4 similarity > 0.9) is present in the cell line's training set, the **CMap** and the **Chembl** datasets will be screened for chemical structures that affect similar BPs to the query.
2. If no structural analogue exists, the appropriate training set from the **CMap** dataset will be screened.

**NOTE** : Our trained deepSIBA ensembles allow query structures of up to 60 atoms. 

In [10]:
model_params = {
    "max_atoms" : int(60), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.25,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.25, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 ,"ConGauss":False
}

In [11]:
screening_params = {
    "query_smile" : "CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1", 
    "cell_line" : "a375", "split" : "5_fold_cv_split" ,"database" : ["Chembl","CMap"],
    "output_dir" : "C:/Users/user/Documents/deepSIBA/results/screening_test1" , "model_path" : "", 
    "atom_limit" : int(60), "N_models" : int(10),
    "name_pattern" : "siam_no_augment", "fold_id" : int(0),
    "screening_threshold" : 0.22
}

The screening_params dictionary contains the parameters required to screen a database with deepSIBA:

1. **query_smile** is the smile string of the chemical structure.
2. **cell_line** is the cellular model of choice out of **(a375,pc3,vcap,mcf7)** for which we have enough available data. Later a merged option will be added.
3. **split** is one of **(train_test_split,5_fold_cv_split,alldata,custom)**. The split selected defines the trained model ensemble that will be loaded. For the screening application the **alldata** split is suggested, where models are trained on the entirety of available data. If **custom** is selected the user must provide a path in **model_path** to load the custom trained models (up to models/ directory).
3. **database** is a list of the supported screening databases. So far only **Chembl** and **CMap** are supported.
4. **output_dir** full path to the desired output directory to write results. The Chembl screening is performed in parts due to the size of the database.
5. **atom_limit** the specified model_params of the trained models, when the split is not **custom** these should be 60.
6. **N_ensemble** is the number of total already trained models and at the same time the models included in the ensembled prediction.
7. **name_pattern** is the pattern of the name of files of models' saved weights. **For example** if the weights are saved in files with names such as **siam_no_augment_18.h5** the **pattern is siam_no_augment** .
8. **fold_id** is an integer, if split == 5_fold_cv_split the fold_id should be 0,1,2,3 or 4 (one less than the corresponding folder's name for this fold), in any other cases the fold_id is not used
9. **screening_threshold** only keep hits with predicted distance less than this threshold in the results.

In [ ]:
from deepSIBA_screening import siba_screening
siba_screening(screening_params, model_params)